In [2]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 
    if image is None:
        raise ValueError(f"Image at {image_path} could not be loaded.")
    image = cv2.resize(image, (128, 128))  
    return image


def extract_hog_features(image):
    features, hog_image = hog(image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True)
    return features


def load_dataset(train_dir):
    X = []
    y = []
    label_map = {'cat': 0, 'dog': 1}

    for img_name in os.listdir(train_dir):
        img_path = os.path.join(train_dir, img_name)
        if 'cat' in img_name:
            label = label_map['cat']
        elif 'dog' in img_name:
            label = label_map['dog']
        else:
            continue  
        
        img = preprocess_image(img_path)
        features = extract_hog_features(img)
        X.append(features)
        y.append(label)
    
    return np.array(X), np.array(y)


train_dir = 'train'


X, y = load_dataset(train_dir)


scaler = StandardScaler()
X = scaler.fit_transform(X)


pca = PCA(n_components=50)  
X_pca = pca.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)


model = LinearSVC()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)



c:\Users\green\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 66.81%
Confusion Matrix:
[[1616  898]
 [ 761 1724]]


c:\Users\green\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [3]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.68      0.64      0.66      2514
           1       0.66      0.69      0.68      2485

    accuracy                           0.67      4999
   macro avg       0.67      0.67      0.67      4999
weighted avg       0.67      0.67      0.67      4999



In [6]:
def predict_new_image(image_path):
    new_image = preprocess_image(image_path)
    new_features = extract_hog_features(new_image)
    new_features = scaler.transform([new_features])
    new_features = pca.transform(new_features)
    prediction = model.predict(new_features)
    if prediction[0]==0:
      label = 'Cat' 
    else :
       label='Dog'
    return label


new_image_path = 'kitty.jpeg'  
label = predict_new_image(new_image_path)
print(f'The image is a: {label}')

The image is a: Cat
